### Попробуем кластеризовать веса не дистилированной модели Bert

*Реализация методов кластеризации "из коробки" была найдена только в TensorFlow, поэтому загрузим tf модель Bert*

In [1]:
import tensorflow as tf
from transformers import pipeline
from transformers import BertTokenizer, TFBertModel

# Load the pretrained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)

# Convert the BERT model to float32 if you haven't already
# bert_model = bert_model.build()

2023-09-21 08:03:06.974659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 08:03:07.102039: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-21 08:03:07.627087: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-09-21 08:03:07.627184: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

In [59]:
bert_model.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


In [60]:
bert_model.weights

[<tf.Variable 'tf_bert_model/bert/embeddings/word_embeddings/weight:0' shape=(30522, 768) dtype=float32, numpy=
 array([[-0.01018257, -0.06154883, -0.02649689, ..., -0.01985357,
         -0.03720997, -0.00975152],
        [-0.01170495, -0.06002603, -0.03233192, ..., -0.01681456,
         -0.04009988, -0.0106634 ],
        [-0.01975381, -0.06273633, -0.03262176, ..., -0.01650258,
         -0.04198876, -0.00323178],
        ...,
        [-0.02176224, -0.0556396 , -0.01346345, ..., -0.00432698,
         -0.0151355 , -0.02489496],
        [-0.04617237, -0.05647721, -0.00192082, ...,  0.01568751,
         -0.01387033, -0.00945213],
        [ 0.00145601, -0.08208051, -0.01597912, ..., -0.00811687,
         -0.04746607,  0.07527421]], dtype=float32)>,
 <tf.Variable 'tf_bert_model/bert/embeddings/token_type_embeddings/embeddings:0' shape=(2, 768) dtype=float32, numpy=
 array([[ 0.00043164,  0.01098826,  0.00370439, ..., -0.00661185,
         -0.00336983, -0.00864201],
        [ 0.00111319, -0.

In [ ]:
import tensorflow_model_optimization as tfmot

In [32]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 4,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

clustered_model = cluster_weights(bert_model, **clustering_params)

ValueError: Arguments clone_function and input_tensors are only supported for Sequential models or Functional models. Received model of type 'TFBertModel', with clone_function=<function _cluster_weights.<locals>._add_clustering_wrapper at 0x7fe55928a160> and input_tensors=None

### Кто разберется с этой ошибкой тот молодец, но сходу не заработало
### Поэтому разеберм пример из документации Tensorflow

In [2]:
import sys

import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot

from model_profiler import model_profiler

In [3]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

# модель с большим количеством слоев
# model = tf.keras.Sequential([
#     tf.keras.layers.InputLayer(input_shape=(28, 28)),
#     tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
#     tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu),
#     tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation=tf.nn.relu),
#     tf.keras.layers.Dropout(0.5),  # Dropout for regularization
#     tf.keras.layers.Dense(10, activation=tf.nn.softmax)  # Output layer
# ])

model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3),
                         activation=tf.nn.relu),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
        layer.set_weights([tf.constant(w, dtype=tf.float32) for w in layer.get_weights()])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 12)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2028)              0         
                                                                 
 dense (Dense)               (None, 10)                20290     
                                                                 
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
____________________________________________________

In [7]:
print("Запомним исходные характеристики модели")
print(model_profiler(model, 12800))

Запомним исходные характеристики модели
| Model Profile                    | Value   | Unit    |
|----------------------------------|---------|---------|
| Selected GPUs                    | ['0']   | GPU IDs |
| No. of FLOPs                     | 0.0     | BFLOPs  |
| GPU Memory Requirement           | 0.6181  | GB      |
| Model Parameters                 | 0.0204  | Million |
| Memory Required by Model Weights | 0.0779  | MB      |


2023-09-21 08:19:53.458438: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-21 08:19:53.458646: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-21 08:19:53.458761: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-21 08:19:53.458914: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-21 08:19:53.459032: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [5]:
model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=10
)

Epoch 1/10


2023-09-20 15:08:33.295060: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500


  53/1688 [..............................] - ETA: 3s - loss: 1.5462 - accuracy: 0.5908 

2023-09-20 15:08:33.591374: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1688/1688 [==============================] - 5s 3ms/step - loss: 0.3025 - accuracy: 0.9144 - val_loss: 0.1238 - val_accuracy: 0.9663
Epoch 2/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.1260 - accuracy: 0.9633 - val_loss: 0.0948 - val_accuracy: 0.9747
Epoch 3/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0876 - accuracy: 0.9747 - val_loss: 0.0684 - val_accuracy: 0.9812
Epoch 4/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0694 - accuracy: 0.9796 - val_loss: 0.0665 - val_accuracy: 0.9803
Epoch 5/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0591 - accuracy: 0.9824 - val_loss: 0.0617 - val_accuracy: 0.9828
Epoch 6/10
1688/1688 [==============================] - 3s 2ms/step - loss: 0.0521 - accuracy: 0.9840 - val_loss: 0.0606 - val_accuracy: 0.9842
Epoch 7/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0461 - accuracy: 0.9860 - val_loss: 0.0618 - val_accuracy: 0.9827
Epo

KeyboardInterrupt: 

In [6]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

Baseline test accuracy: 0.9819999933242798


In [9]:
def print_model_weights_sparsity(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            if "kernel" not in weight.name or "centroid" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )

In [10]:
# Clustering
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 4,
  'cluster_centroids_init': CentroidInitialization.DENSITY_BASED
}

clustered_model = cluster_weights(tf.keras.models.clone_model(model), **clustering_params)

clustered_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# print('Train clustering model:')
# clustered_model.fit(train_images, train_labels,epochs=3, validation_split=0.1)


# Sparsity preserving clustering
# from tensorflow_model_optimization.python.core.clustering.keras.experimental import (
#     cluster,
# )

# cluster_weights = cluster.cluster_weights

# clustering_params = {
#   'number_of_clusters': 4,
#   'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS,
#   'preserve_sparsity': True
# }

# sparsity_clustered_model = cluster_weights(tf.keras.models.clone_model(model), **clustering_params)

# sparsity_clustered_model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# print('Train sparsity preserving clustering model:')
# sparsity_clustered_model.fit(train_images, train_labels,epochs=3, validation_split=0.1)

In [11]:
print("Посмотрим еще методы кластеризации")
dir(CentroidInitialization)

Посмотрим еще методы кластеризации


['DENSITY_BASED',
 'KMEANS_PLUS_PLUS',
 'LINEAR',
 'RANDOM',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [15]:
for w in clustered_model.weights:
    print(w.name, w.numpy())

conv2d/bias:0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
conv2d/kernel:0 [[[[ 0.07660307  0.12273468 -0.06142938  0.17607333  0.11364056
     0.11767642  0.18611585 -0.15826045 -0.125109    0.00971037
    -0.04011129 -0.18119936]]

  [[-0.03982621  0.14150418 -0.0269762  -0.22008401 -0.04034717
    -0.10372871  0.17637368 -0.09620148 -0.02320406 -0.06147139
    -0.05611746 -0.13138255]]

  [[ 0.17687471 -0.05134243 -0.04400572 -0.17912695 -0.12050912
    -0.09690137  0.06401242  0.08343916  0.21745469  0.16980375
    -0.01666982 -0.12173056]]]


 [[[ 0.13791381 -0.18763694  0.0445898  -0.20100786 -0.05598275
    -0.18692598 -0.17226684  0.10692187  0.20862864 -0.04882848
     0.12554304  0.12172069]]

  [[ 0.14717017 -0.00326797  0.10355984 -0.17264882 -0.17829533
     0.04408698 -0.05303565  0.01500793 -0.05874014 -0.21980283
     0.08527033 -0.04704575]]

  [[-0.22596003 -0.22339329 -0.19386666  0.01653345  0.1763468
     0.11823429  0.03739719 -0.2213472  -0.01616766 -0.09359623
     0.1

In [73]:
profile = model_profiler(clustered_model, 48)

print(profile)

| Model Profile                    | Value   | Unit    |
|----------------------------------|---------|---------|
| Selected GPUs                    | ['0']   | GPU IDs |
| No. of FLOPs                     | 0.0     | BFLOPs  |
| GPU Memory Requirement           | 0.0024  | GB      |
| Model Parameters                 | 0.0408  | Million |
| Memory Required by Model Weights | 0.1557  | MB      |


2023-09-20 14:31:43.048305: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-20 14:31:43.048511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-20 14:31:43.048626: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-20 14:31:43.048773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-20 14:31:43.048889: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [13]:
_, clustered_model_accuracy = clustered_model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', clustered_model_accuracy)

2023-09-21 08:21:39.270360: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500


Baseline test accuracy: 0.11349999904632568


In [14]:
profile = model_profiler(clustered_model, 48)

print(profile)

| Model Profile                    | Value   | Unit    |
|----------------------------------|---------|---------|
| Selected GPUs                    | ['0']   | GPU IDs |
| No. of FLOPs                     | 0.0     | BFLOPs  |
| GPU Memory Requirement           | 0.0024  | GB      |
| Model Parameters                 | 0.0408  | Million |
| Memory Required by Model Weights | 0.1557  | MB      |


2023-09-21 08:21:42.557723: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
/usr/local/lib/python3.8/dist-packages/model_profiler/profiler.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.asarray(values).reshape(-1,1),
2023-09-21 08:21:42.557929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-21 08:21:42.558045: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA no

In [15]:
print("Clustered Model sparsity:\n")
print_model_weights_sparsity(clustered_model)
# print("\nSparsity preserved clustered Model sparsity:\n")
# print_model_weights_sparsity(sparsity_clustered_model)

Clustered Model sparsity:

conv2d/kernel:0: 0.00% sparsity  (0/108)
dense/kernel:0: 0.00% sparsity  (0/20280)


In [104]:
clustered_model_file = 'clustered_model.h5'

# Save the model.
clustered_model.save(clustered_model_file)

### Созраним кластеризованную модель

In [110]:
clustered_model = tfmot.clustering.keras.strip_clustering(clustered_model)

clustered_model.save_weights('clustered_model_weights.h5')

In [111]:
clustered_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 13, 13, 12)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 2028)              0         
                                                                 
 dense_4 (Dense)             (None, 10)                20290     
                                                                 
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
__________________________________________________

In [114]:
clustered_model.save(clustered_model_file)

### Загрузим и убедимся, что в значениях весов храянятся только значения центроидов

In [115]:
model = tf.keras.models.load_model("clustered_model.h5")

In [118]:
model.weights

[<tf.Variable 'conv2d_4/kernel:0' shape=(3, 3, 1, 12) dtype=float32, numpy=
 array([[[[-0.8018226 ,  0.2872656 , -0.09660216,  0.2872656 ,
            0.2872656 ,  0.2872656 , -0.09660216, -0.09660216,
            0.2872656 ,  0.8301938 ,  0.2872656 , -0.09660216]],
 
         [[-0.09660216, -0.09660216, -0.09660216,  0.2872656 ,
           -0.09660216,  0.8301938 ,  0.2872656 , -0.09660216,
            0.2872656 , -0.09660216, -0.09660216, -0.09660216]],
 
         [[ 0.2872656 ,  0.2872656 , -0.09660216,  0.2872656 ,
           -0.09660216,  0.2872656 , -0.09660216, -0.09660216,
            0.2872656 , -0.8018226 ,  0.2872656 ,  0.2872656 ]]],
 
 
        [[[-0.09660216,  0.2872656 , -0.09660216, -0.09660216,
            0.2872656 , -0.8018226 , -0.09660216,  0.2872656 ,
            0.2872656 ,  0.8301938 , -0.09660216,  0.2872656 ]],
 
         [[ 0.2872656 , -0.09660216,  0.2872656 ,  0.2872656 ,
            0.2872656 , -0.09660216,  0.2872656 ,  0.2872656 ,
            0.2872656 ,

In [121]:
profile = model_profiler(model, 48)

print(profile)

| Model Profile                    | Value   | Unit    |
|----------------------------------|---------|---------|
| Selected GPUs                    | ['0']   | GPU IDs |
| No. of FLOPs                     | 0.0     | BFLOPs  |
| GPU Memory Requirement           | 0.0023  | GB      |
| Model Parameters                 | 0.0204  | Million |
| Memory Required by Model Weights | 0.0779  | MB      |


/usr/local/lib/python3.8/dist-packages/model_profiler/profiler.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.asarray(values).reshape(-1,1),
2023-09-20 14:58:07.181932: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-20 14:58:07.182143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-20 14:58:07.182260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA no

In [122]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [123]:
_, clustered_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Clustered test accuracy:', clustered_model_accuracy)

Sparsity test accuracy: 0.9556999802589417
